In [ ]:
import numpy
import sys
import torch
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if device == 'cuda':
    torch.backends.cudnn.benchmark = False
    torch.cuda.manual_seed_all(SEED)
print(device)

In [ ]:
file = open("../input/lyrics-with-sentiment/positive_pop.txt").read()

In [ ]:
import string 
    
# Storing the sets of punctuation,
# digits, ascii_letters and whitespace
# in variable result 
result = string.ascii_lowercase + '\',. !?Ж' # characters we want to keep

def check_chars(token):
    return all([ch in result for ch in token])

print(check_chars('hello'))
print(check_chars('\''))
        

In [ ]:
def tokenize_words(text):
    # lowercase everything to standardize it
    text = text.lower()
    text = text.replace('\n', ' Ж ')

    # instantiate the tokenizer
    tokenizer = RegexpTokenizer(r'\w+|\$[\d\.]+|\S+')
    tokens = tokenizer.tokenize(text)

    return " ".join([t for t in tokens if check_chars(t)])

In [ ]:
processed_inputs = tokenize_words(file) # read data
print(len(processed_inputs))

In [ ]:
processed_inputs = processed_inputs[:2500000] # to test with smaller size

In [ ]:
chars = sorted(list(set(processed_inputs))) # list of unique characters
char_to_num = dict((c, i) for i, c in enumerate(chars)) $dictionary with index to word

In [ ]:
input_len = len(processed_inputs)
vocab_len = len(chars)
print ("Total number of characters:", input_len)
print ("Total vocab:", vocab_len)

In [ ]:
seq_length = 30 # legnth of chunk
x_data = []
y_data = []

In [ ]:
# loop through inputs, start at the beginning and go until we hit
# the final character we can create a sequence out of
for i in range(0, input_len - seq_length, 1):
    # Define input and output sequences
    # Input is the current character plus desired sequence length
    in_seq = processed_inputs[i:i + seq_length]

    # Out sequence is the initial character plus total sequence length
    out_seq = processed_inputs[i + seq_length]

    # We now convert list of characters to integers based on
    # previously and add the values to our lists
    x_data.append([char_to_num[char] for char in in_seq])
    y_data.append(char_to_num[out_seq])

In [ ]:
n_patterns = len(x_data) # total amount of chunk
print ("Total Patterns:", n_patterns)

In [ ]:
#pu tin to right format
X = numpy.reshape(x_data, (n_patterns, seq_length, 1)) 
X = X/float(vocab_len)

In [ ]:
y = np_utils.to_categorical(y_data)

In [ ]:
#create a LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(128))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam') # add loss  and optimizer

In [ ]:
#save checkpoints
filepath = "model_weights_saved_pos.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
desired_callbacks = [checkpoint]

In [ ]:
model.fit(X, y, epochs=30, batch_size=256, callbacks=desired_callbacks) #train model

In [ ]:
#read model
filename = "model_weights_saved_pos.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [ ]:
num_to_char = dict((i, c) for i, c in enumerate(chars)) # dictionary for characters

In [ ]:
start = numpy.random.randint(0, len(x_data) - 1) # pick random chunk for testing
pattern = x_data[start]
print("Random Seed:")
print("\"", ''.join([num_to_char[value] for value in pattern]), "\"")

In [ ]:
newl_index = text_to_char('Ж') # add NEWLINE token to text
app_ind = text_to_char('\'')
print(newl_index,app_ind)

In [ ]:
transkeys = num_to_char.keys()
rev_dict = {}
for key in transkeys:
    val = num_to_char[key]
    rev_dict[val] = key
def text_to_char(text):
    # helper functgion that transfors text into the right word indexes
    return [rev_dict[char] for char in text]

In [ ]:
def top_k_sampling(conditional_probability, target_words, k):
    """
    Top_k sampling made for Ngram word generation. Takes a probability distribution for a 
    ngram and returns one out of the top k most probable words.
    """
    n_conds = len(conditional_probability)
    #conditional_probability = np.array(conditional_probability)
    k = min(n_conds,k)
    top_k_probabilities, top_k_indices= tf.math.top_k(conditional_probability, k=k, sorted=True)
    top_k_indices = np.asarray(top_k_indices).astype("int32")
    top_k_redistributed_probability=tf.nn.softmax(np.log(top_k_probabilities))
    top_k_redistributed_probability = np.asarray(top_k_redistributed_probability).astype("float32")
    sampled_index = np.random.choice(top_k_indices, p=top_k_redistributed_probability)
    sampled_token = target_words[sampled_index]
    return sampled_token

In [ ]:
def generate_text(pattern, translate_dict, token_count: int, k = 5):
    """
    Function to generate text with top k sampling for a character based lstm model. 
    Works in a similar fashion to the other text generation functions
    """
    init_string = [num_to_char[c] for c in pattern]
    sys.stdout.write(''.join(init_string))
    for _ in range(token_count):
        
        x = numpy.reshape(pattern, (1, len(pattern), 1))
        x = x / float(vocab_len)
        prediction = model.predict(x, verbose=0)
        prediction = prediction.flatten()
        
        index, result = top_k_sampling(prediction, translate_dict, k )
        if index == newl_index:
            result = '\n'
        sys.stdout.write(result)
        pattern.append(index)
        if len(pattern) > 30: # keep max length op chunks to 30 characters
            pattern = pattern[1:len(pattern)]
